# 🚀 PPO Ultimate Training - Gold Trading AI

**Notebook optimizado para entrenamientos largos sin desconexiones**

---

## ✅ Características:
- Anti-desconexión mejorado (múltiples métodos)
- Entrenamiento en disco LOCAL (evita límites de Google Drive)
- Copia automática a Drive al finalizar
- Checkpoints cada 200k timesteps

---

## ⚙️ Setup:
1. **GPU**: Runtime → Change runtime type → **GPU (A100 recomendado)**
2. **Ejecutar celdas en orden**
3. **IMPORTANTE**: Ejecuta el código anti-desconexión en la consola del navegador (F12)

## 0️⃣ ANTI-DESCONEXIÓN (MUY IMPORTANTE)

### Método 1: Código en Consola del Navegador (MÁS EFECTIVO)

1. Presiona **F12** para abrir DevTools
2. Ve a la pestaña **Console**
3. Pega este código y presiona Enter:

```javascript
function ClickConnect() {
    console.log('🔄 Keeping alive... ' + new Date().toLocaleTimeString());
    
    // Método 1: Click en botón de conexión
    var connectBtn = document.querySelector('#top-toolbar > colab-connect-button');
    if (connectBtn && connectBtn.shadowRoot) {
        var innerBtn = connectBtn.shadowRoot.querySelector('#connect');
        if (innerBtn) innerBtn.click();
    }
    
    // Método 2: Simular actividad
    document.dispatchEvent(new MouseEvent('mousemove'));
    document.dispatchEvent(new KeyboardEvent('keypress'));
}
setInterval(ClickConnect, 60000);
console.log('✅ Anti-desconexión activado!');
```

### Método 2: Ejecuta la siguiente celda (backup)

In [ ]:
# ============================================
# ANTI-DESCONEXIÓN BACKUP (Python)
# ============================================
import IPython
from IPython.display import display, Javascript

# Script mejorado basado en investigación 2024-2025
js_code = '''
function ClickConnect() {
    console.log("🔄 Keep-alive: " + new Date().toLocaleTimeString());
    
    // Intentar múltiples selectores
    var selectors = [
        "colab-connect-button",
        "#top-toolbar > colab-connect-button",
        ".connect-button"
    ];
    
    for (var i = 0; i < selectors.length; i++) {
        var btn = document.querySelector(selectors[i]);
        if (btn) {
            if (btn.shadowRoot) {
                var innerBtn = btn.shadowRoot.querySelector("#connect");
                if (innerBtn) { innerBtn.click(); break; }
            } else {
                btn.click(); break;
            }
        }
    }
    
    // Simular actividad del usuario
    document.dispatchEvent(new MouseEvent("mousemove", {bubbles: true}));
}

// Ejecutar cada 45 segundos (más agresivo)
setInterval(ClickConnect, 45000);
ClickConnect(); // Ejecutar inmediatamente
console.log("✅ Anti-desconexión Python activado!");
'''

display(Javascript(js_code))
print("✅ Anti-desconexión activado desde Python!")
print("")
print("⚠️  IMPORTANTE: Para máxima efectividad, también ejecuta el código")
print("   en la consola del navegador (F12 → Console)")
print("")
print("📌 Puedes minimizar esta pestaña, pero NO la cierres.")

## 1️⃣ Montar Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 2️⃣ Copiar Proyecto a Disco LOCAL (Evita límites de Drive)

**¿Por qué?** Google Drive tiene límites de operaciones. Entrenar directamente en Drive causa errores de cuota. La solución es:
1. Copiar proyecto a disco local de Colab (rápido, sin límites)
2. Entrenar localmente
3. Copiar solo el modelo final a Drive

In [ ]:
import os
import shutil

# Rutas
DRIVE_PATH = '/content/drive/MyDrive/XAUX'
LOCAL_PATH = '/content/XAUX_local'

# Verificar que existe en Drive
if not os.path.exists(DRIVE_PATH):
    print(f"❌ ERROR: No existe la ruta: {DRIVE_PATH}")
    print("\n📁 Carpetas disponibles en tu Drive:")
    !ls /content/drive/MyDrive/
else:
    # Copiar a disco local
    print(f"📂 Copiando proyecto a disco local...")
    print(f"   Origen: {DRIVE_PATH}")
    print(f"   Destino: {LOCAL_PATH}")
    
    if os.path.exists(LOCAL_PATH):
        shutil.rmtree(LOCAL_PATH)
    
    shutil.copytree(DRIVE_PATH, LOCAL_PATH)
    
    os.chdir(LOCAL_PATH)
    print(f"\n✅ Proyecto copiado a disco local!")
    print(f"📂 Directorio actual: {os.getcwd()}")
    print(f"\n💡 El entrenamiento será MUCHO más rápido aquí.")
    print(f"   Los checkpoints se guardarán localmente.")
    print(f"   Al final, copiaremos el modelo a Drive.")

## 3️⃣ Instalar Dependencias

In [ ]:
# PyTorch con CUDA
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# Stable Baselines 3 (PPO)
!pip install -q stable-baselines3[extra]

# Otras dependencias
!pip install -q pandas numpy tqdm matplotlib seaborn gymnasium
!pip install -q yfinance

print("\n✅ Dependencias instaladas!")

## 4️⃣ Verificar GPU

In [ ]:
import torch

print(f"🖥️  PyTorch: {torch.__version__}")
print(f"🎮 CUDA disponible: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"🚀 GPU: {gpu_name}")
    print(f"💾 Memoria: {gpu_memory:.2f} GB")
    device = 'cuda'
else:
    print("⚠️  GPU no disponible, usando CPU (será lento)")
    device = 'cpu'

print(f"\n✅ Device: {device}")

## 5️⃣ Configuración de Entrenamiento

In [ ]:
# ========================================
# CONFIGURACIÓN DE ENTRENAMIENTO
# ========================================

TIMESTEPS = 3_000_000  # 3M recomendado
WINDOW = 128  # 10.6 horas de contexto
DEVICE = device

print("⚙️  Configuración:")
print("="*50)
print(f"📊 Timesteps: {TIMESTEPS:,}")
print(f"🪟 Window: {WINDOW} barras ({WINDOW * 5 / 60:.1f} horas)")
print(f"🖥️  Device: {DEVICE}")
print("="*50)

# Estimación de tiempo
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    if 'A100' in gpu_name:
        hours = TIMESTEPS / 1_000_000 * 3
    elif 'V100' in gpu_name or 'T4' in gpu_name:
        hours = TIMESTEPS / 1_000_000 * 5
    else:
        hours = TIMESTEPS / 1_000_000 * 4
    print(f"\n⏱️  Tiempo estimado: ~{hours:.1f} horas en {gpu_name}")
else:
    print("\n⚠️  Sin GPU, el entrenamiento será muy lento")

## 6️⃣ Entrenar PPO (Disco Local)

**El entrenamiento corre en disco local = Sin límites de Drive!**

In [ ]:
!python train/train_ppo_ultimate.py \
    --timesteps {TIMESTEPS} \
    --window {WINDOW} \
    --device {DEVICE}

## 7️⃣ Copiar Resultados a Google Drive

**IMPORTANTE**: Ejecuta esta celda cuando termine el entrenamiento para guardar tu modelo en Drive.

In [ ]:
import shutil
import os
import glob

# Rutas
LOCAL_RESULTS = '/content/XAUX_local/train/ppo_ultimate'
DRIVE_RESULTS = '/content/drive/MyDrive/XAUX/train/ppo_ultimate'

# Crear directorio en Drive si no existe
os.makedirs(DRIVE_RESULTS, exist_ok=True)

print("📤 Copiando resultados a Google Drive...")
print(f"   Origen: {LOCAL_RESULTS}")
print(f"   Destino: {DRIVE_RESULTS}")

# Copiar archivos importantes
files_to_copy = [
    'ppo_ultimate_final.zip',
    'ppo_ultimate_metrics.json',
    'vec_normalize.pkl'
]

copied = 0
for filename in files_to_copy:
    src = os.path.join(LOCAL_RESULTS, filename)
    if os.path.exists(src):
        dst = os.path.join(DRIVE_RESULTS, filename)
        shutil.copy2(src, dst)
        size_mb = os.path.getsize(src) / 1024 / 1024
        print(f"   ✅ {filename} ({size_mb:.1f} MB)")
        copied += 1

# También copiar último checkpoint
checkpoints = sorted(glob.glob(f'{LOCAL_RESULTS}/*.zip'))
if checkpoints:
    latest_cp = checkpoints[-1]
    dst = os.path.join(DRIVE_RESULTS, os.path.basename(latest_cp))
    if not os.path.exists(dst):
        shutil.copy2(latest_cp, dst)
        print(f"   ✅ {os.path.basename(latest_cp)} (último checkpoint)")
        copied += 1

print(f"\n🎉 {copied} archivos copiados a Drive!")
print(f"\n📂 Tus resultados están en:")
print(f"   {DRIVE_RESULTS}")

## 8️⃣ Ver Resultados

In [ ]:
import json

# Intentar cargar desde local primero, luego desde Drive
metrics_paths = [
    '/content/XAUX_local/train/ppo_ultimate/ppo_ultimate_metrics.json',
    '/content/drive/MyDrive/XAUX/train/ppo_ultimate/ppo_ultimate_metrics.json'
]

metrics = None
for path in metrics_paths:
    if os.path.exists(path):
        with open(path, 'r') as f:
            metrics = json.load(f)
        break

if metrics:
    print("="*70)
    print("📊 RESULTADOS PPO ULTIMATE")
    print("="*70)
    
    print(f"\n🎯 VALIDATION (2022):")
    val = metrics['validation']
    print(f"   Total Return: {val['total_return']:.2f}%")
    print(f"   Sharpe Ratio: {val['sharpe_ratio']:.2f}")
    print(f"   Max Drawdown: {val['max_drawdown']:.2f}%")
    print(f"   Win Rate: {val['win_rate']:.1f}%")
    print(f"   Trades: {val['trades']}")
    
    print(f"\n🎯 TEST (2023-2025):")
    test = metrics['test']
    print(f"   Total Return: {test['total_return']:.2f}%")
    print(f"   Sharpe Ratio: {test['sharpe_ratio']:.2f}")
    print(f"   Max Drawdown: {test['max_drawdown']:.2f}%")
    print(f"   Win Rate: {test['win_rate']:.1f}%")
    print(f"   Trades: {test['trades']}")
else:
    print("❌ Métricas no encontradas. Ejecuta el entrenamiento primero.")

## 9️⃣ Resumir Entrenamiento (si se interrumpe)

Si Colab se desconecta, puedes continuar desde el último checkpoint.

In [ ]:
import glob
import os

# Buscar checkpoints en ambas ubicaciones
local_dir = '/content/XAUX_local/train/ppo_ultimate'
drive_dir = '/content/drive/MyDrive/XAUX/train/ppo_ultimate'

for check_dir in [local_dir, drive_dir]:
    if os.path.exists(check_dir):
        checkpoints = sorted(glob.glob(f'{check_dir}/*.zip'))
        if checkpoints:
            print(f"📁 Checkpoints en: {check_dir}")
            for cp in checkpoints[-3:]:
                print(f"   • {os.path.basename(cp)}")
            print(f"\n🔄 Para resumir, ejecuta:")
            print(f"")
            print(f"CHECKPOINT = '{checkpoints[-1]}'")
            print(f"!python train/train_ppo_ultimate.py --timesteps {TIMESTEPS} --window {WINDOW} --device {DEVICE} --resume $CHECKPOINT")
            break
else:
    print("⚠️ No hay checkpoints aún")

---

## 📝 Tips para Evitar Desconexiones

1. **Ejecuta el código JavaScript en F12** (más efectivo que Python)
2. **Mantén la pestaña visible** (no en background)
3. **Usa Chrome** (mejor compatibilidad)
4. **Revisa cada 1-2 horas** por si aparece CAPTCHA
5. **Los checkpoints están en disco local** - si se desconecta, el progreso se pierde
6. **Ejecuta la celda de copiar a Drive** regularmente si quieres backup

---

🎉 **¡Listo! El entrenamiento local es más rápido y sin límites de Drive.**